<a href="https://colab.research.google.com/github/zhangou888/NN/blob/main/Step-by-step%20PyTorch%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-by-step PyTorch Neural Network (Colab-Ready)

In [ ]:
# Step 1: Install dependencies (if not already in Colab)
# Uncomment the line below if you're running outside Colab
# !pip install torch torchvision
